In [1]:
!pip install transformers[torch] datasets evaluate optuna

In [2]:
!pip install -U datasets

In [3]:
import pandas as pd
import numpy as np
import json
from joblib import dump, load

import os
import random
from tqdm.notebook  import tqdm

import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader

from transformers import (AutoConfig,
                          AutoTokenizer,
                          DefaultDataCollator,
                          AutoModelForQuestionAnswering,
                          TrainingArguments,
                          Trainer,
                          default_data_collator,
                          get_scheduler)

from datasets import load_dataset
import evaluate
import collections
import optuna

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
def seed_everything(seed):
  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = True

seed_everything(42)
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:32"

In [8]:
dataset = load_dataset(path='/kaggle/input',data_files='./custom_data.json',split='train')
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['context', 'answers', 'id', 'question'],
    num_rows: 28024
})

In [9]:
squad = dataset.train_test_split(.2)
squad

DatasetDict({
    train: Dataset({
        features: ['context', 'answers', 'id', 'question'],
        num_rows: 22419
    })
    test: Dataset({
        features: ['context', 'answers', 'id', 'question'],
        num_rows: 5605
    })
})

In [10]:
squad["train"][0]

{'context': '경제협력개발기구(OECD)가 발표한 ‘더 나은 삶 지수(Better Life Index, BLI)’ 중 하나인 ‘공동체(Community)’ 지수에서 대한민국은 러시아와 브라질을 포함한 36개 국가 중 최하위를 기록했다. 한국 사회에서 공동체가 파괴되어온 주요인으로 다양한 의견이 개진되었으나, 한국도시의 오래된 특징인 아파트와 고립된 생활 공간, 차도와 인도의 구분이 없는 길과 건물 배치가 이웃 간 소통을 가로막고 갈등이 생겨도 이를 원활하게 해결할 수 없는 환경을 조성했다는 지적이 주목을 받았다(김재형, 2016). 도움이 필요할 때 이웃의 관심과 손길을 기대하기 어려운 환경은 고독사, 아동학대, 가정폭력, 자살 등 다양한 사회문제를 방치하고 악화시키는 데 일조한다. 이러한 공동체 와해와 지역현안 문제를 주민들의 참여를 통해 적극적으로 해결하고자 시행된 정책사업 중 하나가 2010년부터 각 지자체에 본격적으로 도입된 마을 만들기 사업이다. 마을 만들기 사업의 유형은 주체와 내용에 따라 다양한 형태로 발현되지만, 그 핵심은 주민참여를 주요 동력으로 공동의 지역문제와 욕구를 해결하고, 공동체 회복을 통해 주민의 삶의 질을 향상시킨다는 정책적 목적으로 귀결된다(박선희, 2014).',
 'answers': {'answer_start': [84], 'text': ['대한민국']},
 'id': 'QUES_UniBPiuRnf',
 'question': '더 나은 삶 지수 중 하나인 공동체 지수에서 어느 나라가 최하위를 기록했니'}

In [11]:
print("Context: ", squad["train"][0]["context"])
print("Question: ", squad["train"][0]["question"])
print("Answer: ", squad["train"][0]["answers"])

Context:  경제협력개발기구(OECD)가 발표한 ‘더 나은 삶 지수(Better Life Index, BLI)’ 중 하나인 ‘공동체(Community)’ 지수에서 대한민국은 러시아와 브라질을 포함한 36개 국가 중 최하위를 기록했다. 한국 사회에서 공동체가 파괴되어온 주요인으로 다양한 의견이 개진되었으나, 한국도시의 오래된 특징인 아파트와 고립된 생활 공간, 차도와 인도의 구분이 없는 길과 건물 배치가 이웃 간 소통을 가로막고 갈등이 생겨도 이를 원활하게 해결할 수 없는 환경을 조성했다는 지적이 주목을 받았다(김재형, 2016). 도움이 필요할 때 이웃의 관심과 손길을 기대하기 어려운 환경은 고독사, 아동학대, 가정폭력, 자살 등 다양한 사회문제를 방치하고 악화시키는 데 일조한다. 이러한 공동체 와해와 지역현안 문제를 주민들의 참여를 통해 적극적으로 해결하고자 시행된 정책사업 중 하나가 2010년부터 각 지자체에 본격적으로 도입된 마을 만들기 사업이다. 마을 만들기 사업의 유형은 주체와 내용에 따라 다양한 형태로 발현되지만, 그 핵심은 주민참여를 주요 동력으로 공동의 지역문제와 욕구를 해결하고, 공동체 회복을 통해 주민의 삶의 질을 향상시킨다는 정책적 목적으로 귀결된다(박선희, 2014).
Question:  더 나은 삶 지수 중 하나인 공동체 지수에서 어느 나라가 최하위를 기록했니
Answer:  {'answer_start': [84], 'text': ['대한민국']}


In [12]:
train_dataset = squad["train"]
valid_dataset = squad["test"]

In [13]:
# select samples
train_dataset = train_dataset.select(range(1000))
valid_dataset = valid_dataset.select(range(100))

In [14]:
MODEL_NAME ='monologg/kobigbird-bert-base'
config = AutoConfig.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

config.json:   0%|          | 0.00/870 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/241k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/492k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/169 [00:00<?, ?B/s]

In [15]:
max_length = 1024
stride = 128

def preprocess_training_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride = stride,
        return_offsets_mapping=True,
        return_overflowing_tokens=True,
        padding="max_length",
    )

    #the offset mapping gives us a tuple indicating the sub-token’s start position and end position relative to the original token it was split from
    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
      sample_idx = sample_map[i]
      answer = answers[sample_idx]
      start_char = answer["answer_start"][0]
      end_char = answer["answer_start"][0] + len(answer["text"][0])
      sequence_ids = inputs.sequence_ids(i)

      # Find the start and end of the context
      idx = 0
      while sequence_ids[idx] != 1:
          idx += 1
      context_start = idx
      while sequence_ids[idx] == 1:
          idx += 1
      context_end = idx - 1

      # If the answer is not fully inside the context, label it (0, 0)
      if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
          start_positions.append(0)
          end_positions.append(0)
      else:
          # Otherwise it's the start and end token positions
          idx = context_start
          while idx <= context_end and offset[idx][0] <= start_char:
              idx += 1
          start_positions.append(idx - 1)

          idx = context_end
          while idx >= context_start and offset[idx][1] >= end_char:
              idx -= 1
          end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions

    return inputs

In [16]:
def preprocess_validation_function(exmaples):
    questions = [q.strip() for q in exmaples["question"]]
    inputs = tokenizer(
        questions,
        exmaples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(exmaples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [17]:
train_set = train_dataset.map(preprocess_training_function,
                              batched=True,
                              remove_columns= train_dataset.column_names,
                                    load_from_cache_file=False,)

valid_set = valid_dataset.map(preprocess_validation_function,
                              batched=True,
                              remove_columns= valid_dataset.column_names,
                                    load_from_cache_file=False,)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [18]:
train_set

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 1000
})

In [19]:
valid_set

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 100
})

In [20]:
valid_modelset =valid_set.remove_columns(["example_id", "offset_mapping"])

In [21]:
train_set.set_format("torch")
valid_modelset.set_format("torch")

In [22]:
def get_loader(batch_size):
  trainloader = DataLoader(train_set, shuffle=True, batch_size=batch_size, collate_fn=default_data_collator)
  validloader = DataLoader(valid_modelset, shuffle=False, batch_size=batch_size, collate_fn=default_data_collator)
  return trainloader, validloader

In [23]:
metric = evaluate.load("squad")

In [24]:
n_best = 20
max_answer_length = 30

def compute_metrics(start_logits, end_logits,features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    if (end_index < start_index
                        or end_index - start_index +1 > max_answer_length):
                      continue
                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],}
                    answers.append(answer)

        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append({"id": example_id, "prediction_text": best_answer["text"]})
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})
    references = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=references)

In [25]:
model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME)

model.safetensors:   0%|          | 0.00/458M [00:00<?, ?B/s]

Some weights of BigBirdForQuestionAnswering were not initialized from the model checkpoint at monologg/kobigbird-bert-base and are newly initialized: ['qa_classifier.output.dense.bias', 'qa_classifier.output.dense.weight', 'qa_classifier.intermediate.dense.weight', 'qa_classifier.output.LayerNorm.bias', 'qa_classifier.intermediate.dense.bias', 'qa_classifier.qa_outputs.weight', 'qa_classifier.qa_outputs.bias', 'qa_classifier.output.LayerNorm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
accumulation_steps = 8

def train(trainloader, model, optimizer, lr_scheduler, device):
    model.train()
    running_loss = 0.0
    for idx, batch in enumerate(trainloader):
      batch ={k: v.to(device) for k, v in batch.items()}
      outputs = model(**batch)
      loss = outputs.loss
      running_loss += loss.item()
      if (idx +1) % accumulation_steps == 0:
          loss.backward()
          optimizer.step()
          optimizer.zero_grad()
          lr_scheduler.step()
    train_loss = running_loss / len(trainloader)
    return train_loss

In [27]:
def validate(validloader, model, device):
    model.eval()
    start_logits =[]
    end_logits = []
    with torch.no_grad():
        for idx, batch in enumerate(validloader):
            batch ={k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)

            start_logits.append(outputs.start_logits.detach().cpu().numpy())
            end_logits.append(outputs.end_logits.detach().cpu().numpy())

        start_logits = np.concatenate(start_logits, axis=0)
        end_logits = np.concatenate(end_logits, axis=0)
        results = compute_metrics(start_logits, end_logits, valid_set, valid_dataset)
    return  results["exact_match"], results["f1"]

In [28]:
torch.cuda.empty_cache()

def train_mrc(trial):
    Learning_rate=trial.suggest_float('learning_rate', 5e-7, 5e-3, log=True)
    Weight_decay = trial.suggest_float('weight_decay', 0.0, 0.3)
    num_epochs = 10
    Batch_size = 1
    accumulation_steps = 32
    trainloader, validloader = get_loader(Batch_size)
    num_update_steps_per_epoch = len(trainloader)
    num_training_steps = num_epochs * num_update_steps_per_epoch

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME)
    model = model.to(device)

    optimizer = AdamW(model.parameters(), lr=Learning_rate, weight_decay=Weight_decay)
    lr_scheduler = get_scheduler(
      trial.suggest_categorical("SchedulerType", ['linear', 'cosine', 'constant']),
      optimizer=optimizer,
      num_warmup_steps=0,
      num_training_steps=num_training_steps,
      )
    for epoch in tqdm(range(1, num_epochs+1)):
        train_loss = train(trainloader, model, optimizer, lr_scheduler, device)
        em, f1 = validate(validloader, model, device)
        print("Epoch:{:1d}/{:1d}, Train Loss:{:.4f}, EM:{:.2f}, F1:{:.2f}".format(epoch, num_epochs, train_loss, em, f1 ))

    return em


In [29]:
study = optuna.create_study(sampler=optuna.samplers.TPESampler(), direction="maximize")
study.optimize(train_mrc, n_trials=20)

[I 2023-12-17 00:41:16,098] A new study created in memory with name: no-name-1b8823dc-360c-455b-90e3-2d2d7260d0a7
Some weights of BigBirdForQuestionAnswering were not initialized from the model checkpoint at monologg/kobigbird-bert-base and are newly initialized: ['qa_classifier.output.dense.bias', 'qa_classifier.output.dense.weight', 'qa_classifier.intermediate.dense.weight', 'qa_classifier.output.LayerNorm.bias', 'qa_classifier.intermediate.dense.bias', 'qa_classifier.qa_outputs.weight', 'qa_classifier.qa_outputs.bias', 'qa_classifier.output.LayerNorm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch:1/10, Train Loss:5.6749, EM:0.00, F1:2.38


  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch:9/10, Train Loss:0.5133, EM:70.00, F1:77.51


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2023-12-17 09:51:15,398] Trial 19 finished with value: 77.0 and parameters: {'learning_rate': 1.8581330170625473e-05, 'weight_decay': 0.1764120796395892, 'SchedulerType': 'cosine'}. Best is trial 19 with value: 77.0.


Epoch:10/10, Train Loss:0.4567, EM:77.00, F1:81.01


In [35]:
dump(study, '/kaggle/working/aiconnect_mrc_optuna.pkl')

['/kaggle/working/aiconnect_mrc_optuna.pkl']

In [36]:
study = load('/kaggle/working/aiconnect_mrc_optuna.pkl')
df = study.trials_dataframe()
df.head()

,number,value,datetime_start,datetime_complete,duration,params_SchedulerType,params_learning_rate,params_weight_decay,state
0,0,0.0,2023-12-17 00:41:16.100126,2023-12-17 01:08:39.024829,0 days 00:27:22.924703,cosine,0.000775,0.152012,COMPLETE
1,1,69.0,2023-12-17 01:08:39.026562,2023-12-17 01:36:42.748698,0 days 00:28:03.722136,cosine,0.000066,0.016688,COMPLETE
2,2,70.0,2023-12-17 01:36:42.750412,2023-12-17 02:04:32.639160,0 days 00:27:49.888748,constant,0.000010,0.228739,COMPLETE
3,3,68.0,2023-12-17 02:04:32.640706,2023-12-17 02:32:31.928501,0 days 00:27:59.287795,cosine,0.000009,0.267987,COMPLETE
4,4,11.0,2023-12-17 02:32:31.930029,2023-12-17 03:00:31.746113,0 days 00:27:59.816084,linear,0.000165,0.084027,COMPLETE


In [37]:
df.sort_values(by='value',  ascending=False)

,number,value,datetime_start,datetime_complete,duration,params_SchedulerType,params_learning_rate,params_weight_decay,state
19,19,77.0,2023-12-17 09:23:15.862934,2023-12-17 09:51:15.397950,0 days 00:27:59.535016,cosine,1.858133e-05,0.176412,COMPLETE
14,14,76.0,2023-12-17 07:07:15.448408,2023-12-17 07:34:24.196894,0 days 00:27:08.748486,constant,3.338102e-05,0.295558,COMPLETE
7,7,75.0,2023-12-17 03:56:33.683078,2023-12-17 04:24:27.713754,0 days 00:27:54.030676,linear,5.131905e-06,0.295328,COMPLETE
16,16,72.0,2023-12-17 08:01:29.297012,2023-12-17 08:28:44.336121,0 days 00:27:15.039109,constant,3.854111e-05,0.247193,COMPLETE
12,12,72.0,2023-12-17 06:12:54.948784,2023-12-17 06:40:02.737127,0 days 00:27:07.788343,constant,2.187561e-05,0.298273,COMPLETE
2,2,70.0,2023-12-17 01:36:42.750412,2023-12-17 02:04:32.639160,0 days 00:27:49.888748,constant,1.030153e-05,0.228739,COMPLETE
1,1,69.0,2023-12-17 01:08:39.026562,2023-12-17 01:36:42.748698,0 days 00:28:03.722136,cosine,6.553795e-05,0.016688,COMPLETE
3,3,68.0,2023-12-17 02:04:32.640706,2023-12-17 02:32:31.928501,0 days 00:27:59.287795,cosine,9.190383e-06,0.267987,COMPLETE
9,9,68.0,2023-12-17 04:51:34.929503,2023-12-17 05:18:44.711575,0 days 00:27:09.782072,linear,6.681311e-06,0.258194,COMPLETE
11,11,67.0,2023-12-17 05:45:51.355071,2023-12-17 06:12:54.947071,0 days 00:27:03.592000,constant,4.264506e-06,0.211727,COMPLETE


In [55]:
print(f"Number of trials on the Pareto front: {len(study.best_trials)}")

trial_with_highest_em = max(study.best_trials, key=lambda t: t.values)
print(f"Trial with highest accuracy: ")
print(f"\tnumber: {trial_with_highest_em.number}")
print(f"\tparams: {trial_with_highest_em.params}")
print(f"\tvalues: {trial_with_highest_em.values}")

Number of trials on the Pareto front: 1
Trial with highest accuracy: 
	number: 19
	params: {'learning_rate': 1.8581330170625473e-05, 'weight_decay': 0.1764120796395892, 'SchedulerType': 'cosine'}
	values: [77.0]


In [58]:
optuna.visualization.plot_parallel_coordinate(study, target_name="EM Score")

In [70]:
optuna.visualization.plot_optimization_history(study)

In [66]:
optuna.visualization.plot_param_importances(
    study, target_name="EM"
)

In [69]:
optuna.visualization.plot_slice(study, params=["learning_rate", "weight_decay", "SchedulerType"])